In [1]:
import cv2
import numpy as np
import time
import glob
import matplotlib.pyplot as plt
import os
import time

In [2]:
def algo(v_path):
    
    files = glob.glob('02_intermediate/*')
    for f in files:
        os.remove(f)
    
    cap = cv2.VideoCapture("01_org_video/{}".format(v_path))
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print(frame_count)
    
    img_array = []
    for i in range(int(frame_count)):
        _,img = cap.read(i)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        s = int(round(img.shape[0]/600,1)*10)
        filterSize =(s, s) 
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, filterSize) 
        tophat_img = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel) 

        kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]],dtype=int)
        kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]],dtype=int)
        x = cv2.filter2D(tophat_img, cv2.CV_16S, kernelx)
        y = cv2.filter2D(tophat_img, cv2.CV_16S, kernely)
        absX = cv2.convertScaleAbs(x) 
        absY = cv2.convertScaleAbs(y) 
        Prewitt = cv2.addWeighted(absX,0.5,absY,0.5,0)
        
        img_array.append(cv2.cvtColor(Prewitt, cv2.COLOR_GRAY2BGR))
    

    size = (Prewitt.shape[1], Prewitt.shape[0])
    out = cv2.VideoWriter('03_output/V_{}_processed.avi'.format(v_path.split(".")[0]),cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

    video_path = '03_output/V_{}_processed.avi'.format(v_path.split(".")[0])
    cap = cv2.VideoCapture(video_path)
    width =int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height =int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    one_frame = np.zeros((height,width),dtype=np.uint8)
    two_frame = np.zeros((height,width),dtype=np.uint8)
    three_frame = np.zeros((height,width),dtype=np.uint8)

    II = np.zeros((height,width,3),dtype=np.uint8)
    
    for k in range(int(frame_count)):
        ret,frame = cap.read()
        frame_gray =cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        if not ret:
            break
        one_frame,two_frame,three_frame = two_frame,three_frame,frame_gray
        abs1 = cv2.absdiff(one_frame,two_frame)
        _,thresh1 = cv2.threshold(abs1,15,555,cv2.THRESH_BINARY)
        abs2 =cv2.absdiff(two_frame,three_frame)
        _,thresh2 =cv2.threshold(abs2,15,555,cv2.THRESH_BINARY)
        
        binary1 = cv2.bitwise_or(thresh1,thresh2)
        cv2.imwrite("02_intermediate/tt_{}.jpg".format(k), binary1)
    
    for k in range(int(frame_count)):
        II += cv2.imread("02_intermediate/tt_{}.jpg".format(k))
        
    cv2.imwrite("03_output/P_{}.jpg".format(v_path), II)
    cap.release()
    cv2.destroyAllWindows()

In [3]:
v_path = "五矿1.avi"
t1 = time.time()
algo(v_path)
t2 = time.time()
print(t2 - t1)

400.0
2.172616958618164
